## Task 04. Color Object Detection 

Now we have a more complex scenario. We have several objects of the same colour, i.e. 5 yellow balls. They are diposed around the scene having diferent sizes. What we want now is to track the only one that is moving around, no matter its size. 


### Goals:
* Step 1: Basic motion detection 

* Step 2: Detect the presence of colored objects using computer vision techniques.

* Step 3: Track the object as it moves around in the video frames, drawing its previous positions as it moves, creating a tail behind it

In [8]:
# import the necessary packages

from collections import deque
import numpy as np
import cv2
import time
import datetime


#Convenience resize function
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized
VIDEODEV = 0


In [9]:
def setup_trackbars(range_filter):
    cv2.namedWindow("Trackbars", 0)
    for i in ["MIN", "MAX"]:
        for j in range_filter:
            v = 0 if i == "MIN" else 255
            cv2.createTrackbar("%s_%s" % (j, i), "Trackbars", v, 255, lambda x : None)
            
            

def get_trackbar_values(range_filter):
    values = {}

    for i in ["MIN", "MAX"]:
        for j in range_filter:
            v = cv2.getTrackbarPos("%s_%s" % (j, i), "Trackbars")
            values["%s_%s" % (j, i)] = v
    return values

camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
setup_trackbars('HSV')
while True:
        (grabbed, frame) = camera.read()
        frame_to_thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        v = get_trackbar_values('HSV')
        thresh = cv2.inRange(frame_to_thresh, (v['H_MIN'], v['S_MIN'], v['V_MIN']), (v['H_MAX'], v['S_MAX'], v['V_MAX']))
        cv2.imshow("Original", frame)
        cv2.imshow("Thresh", thresh)
        if cv2.waitKey(1) & 0xFF is ord('q'):
            break
            
print "Lower = (%d,%d,%d)" % (v['H_MIN'], v['S_MIN'], v['V_MIN'])
print "Upper = (%d,%d,%d)" % (v['H_MAX'], v['S_MAX'], v['V_MAX'])
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

Lower = (0,108,121)
Upper = (72,255,255)


In [10]:
# define the lower and upper boundaries of the "yellow"
# ball in the HSV color space, then initialize the
# list of tracked points
colorLower = (v['H_MIN'], v['S_MIN'], v['V_MIN'])
colorUpper = (v['H_MAX'], v['S_MAX'], v['V_MAX'])
tailsize = 64
pts = deque(maxlen=tailsize)

Now let's define the color range of our object in the HSV color space. For this we will use a set of trackbars H_Min, H_Max, S_Min, S_Max, V_Min, V_Max for trying to isolate our object. To do that we will use the inRange function which return a binary image with true values where the pixels in the original frame falls between the given range in [HSV](https://en.wikipedia.org/wiki/HSL_and_HSV) color space.

In [11]:
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
# keep looping
while True:
    # grab the current frame
    (grabbed, frame) = camera.read()
    
    # resize the frame, blur it, and convert it to the HSV
    # color space
    frame = resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
     # show the frame to our screen
    cv2.imshow("HSV", hsv)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

We get the reference frame for motion detection.

In [12]:
VIDEODEV = 0
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
time.sleep(0.25)
firstFrame = None

# loop over the frames of the video
while True:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    (grabbed, frame) = camera.read()
    text = "Unoccupied"
 
    # resize the frame, convert it to grayscale, and blur it
    frame = resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
 
    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", gray)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
    
firstFrame = gray
camera.release()
cv2.destroyAllWindows()

We show the color feature detected and the movement if detected inside the circle area.

In [15]:
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
min_area = 500
diffDelta = 25
time.sleep(0.25)
#colorLower = (0,157,109)
#colorUpper = (255,255,255)
# keep looping
while True:
    # grab the current frame
    (grabbed, frame_orig) = camera.read()
    
    # resize the frame, blur it, and convert it to the HSV
    # color space
    frame = resize(frame_orig, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    frame_grey = resize(frame_orig, width=500)
    gray = cv2.cvtColor(frame_grey, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    # construct a mask for the color selected, then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, diffDelta, 255, cv2.THRESH_BINARY)[1]
 
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts_grey = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
 
    # loop over the contours
        #text = "Occupied"


    # find contours in the mask and initialize the current
    # (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None

    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        #if center[0]>x & center[0] < (x+w) & center[1]> y & center[1]
        for cg in cnts_grey:
        # if the contour is too small, ignore it
            if cv2.contourArea(cg) < min_area:
                continue
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
            (x1, y1, w, h) = cv2.boundingRect(cg)
            if x>x1 and x < (x1+w) and y>y1 and y<(y1+h):
                cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        # only proceed if the radius meets a minimum size
        if radius > 10:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            cv2.putText(frame, "x: {}, y: {}".format(center[0], center[1]),(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.35, (0, 0, 255), 1)
    # update the points queue
    pts.appendleft(center)

    # loop over the set of tracked points
    for i in xrange(1, len(pts)):
        # if either of the tracked points are None, ignore
        # them
        if pts[i - 1] is None or pts[i] is None:
            continue

        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        thickness = int(np.sqrt(tailsize / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

         # show the frame and record if the user presses a key
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)

    # show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()